In [2]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

os_env = 'mac'


if os_env == 'window':

    emb_path_trn_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_train_new_version(0313).npy'
    emb_path_vld_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_valid_new_version(0313).npy'

    emb_path_trn_eicu = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_eicu_new_version(0313).npy'

    mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
    if module_path not in sys.path:
        sys.path.append(module_path)

    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
    if module_path not in sys.path:
        sys.path.append(module_path)

else:
    emb_path_trn_mimic = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/Train/result/emb_train_new_version(0313).npy'
    emb_path_vld_mimic = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/Train/result/emb_valid_new_version(0313).npy'

    emb_path_trn_eicu = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/Train/result/emb_eicu_new_version(0313).npy'

    mimic_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/'

    if module_path not in sys.path:
        sys.path.append(module_path)

    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
    if module_path not in sys.path:
        sys.path.append(module_path)

from cohort_loader_new_version0229 import *
import eicu_year_process

import integrating_embedding
import get_hospital_eicu


mimic_train_emb, mimic_valid_emb, event = integrating_embedding.integrating_for_subtask(mimic_path, emb_path_trn_mimic, emb_path_vld_mimic, _,'mimic')
eicu_test_emb, event_eicu = integrating_embedding.integrating_for_subtask(eicu_path, emb_path_trn_eicu, _,_, 'eicu')

In [5]:
eicu_test_emb = eicu_test_emb[eicu_test_emb['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)

In [6]:
module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/'

if module_path not in sys.path:
    sys.path.append(module_path)

import incorperate_subtask_label
from imp import reload
reload(incorperate_subtask_label)

<module 'incorperate_subtask_label' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/incorperate_subtask_label.py'>

In [43]:
# train, val -> Trainset (underfitting 방지)

mort_mimic_train, mort_mimic_valid, mort_eicu_test = incorperate_subtask_label.mortality_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb, os_env='mac')
mimic_mort_trainset = pd.concat([mort_mimic_train, mort_mimic_valid], axis = 0)

los_mimic_train, los_mimic_valid, los_eicu_test = incorperate_subtask_label.LOS_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb, os_env='mac')
mimic_los_trainset = pd.concat([los_mimic_train, los_mimic_valid], axis = 0)

ards8h_mimic_train, ards8h_mimic_valid, ards8h_eicu_test = incorperate_subtask_label.ARDS8_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb, os_env='mac')
mimic_ards8h_trainset = pd.concat([ards8h_mimic_train, ards8h_mimic_valid], axis = 0)

sic8h_mimic_train, sic8h_mimic_valid, sic8h_eicu_test = incorperate_subtask_label.SIC8_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb, os_env='mac')
mimic_sic8h__trainset = pd.concat([sic8h_mimic_train, sic8h_mimic_valid], axis = 0)

In [44]:
ARDS8h_event_train = ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS8h_event_train['ARDS_next_8h']='event'
ARDS8h_event_valid = ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS8h_event_valid['ARDS_next_8h']='event'
ARDS8h_event_test = ards8h_eicu_test[ards8h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS8h_event_test['ARDS_next_8h']='event'

SIC8h_event_train = sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='SIC']
SIC8h_event_train['SIC_next_8h']='event'
SIC8h_event_valid = sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='SIC']
SIC8h_event_valid['SIC_next_8h']='event'
SIC8h_event_test = sic8h_eicu_test[sic8h_eicu_test['Annotation_SIC']=='SIC']
SIC8h_event_test['SIC_next_8h']='event'

In [45]:
import split
from imp import reload
reload(split)

<module 'split' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/split.py'>

In [46]:
X_train_mort, y_train_mort, _ = split.split_X_Y_MORT(mimic_mort_trainset[~(mimic_mort_trainset['death']=='event')], mode = 'mimic')
X_train_los, y_train_los, _ = split.split_X_Y_LOS(mimic_los_trainset, mode = 'mimic')
X_train_ards8h, y_train_ards8h, _ = split.split_X_Y_ARDS8h(mimic_ards8h_trainset, mode = 'mimic')
X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(mimic_sic8h__trainset, mode = 'mimic')

In [47]:
import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [48]:
#mortality

lgbm_wrapper = LGBMClassifier(random_state = 42, verbose=-1)
lgbm_wrapper.fit(X_train_mort, y_train_mort)

X_test_mort, y_test_mort, test_output_mort = split.split_X_Y_MORT(mort_eicu_test[~(mort_eicu_test['death']=='event')], mode = 'eicu')

X_test_mort = X_test_mort.drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)

test_preds_mort = lgbm_wrapper.predict(X_test_mort)
test_output_mort['prediction_label'] = test_preds_mort
test_output_mort['prediction_prob'] = lgbm_wrapper.predict_proba(X_test_mort)[:, 1]

test_eval = Multiclass_evaluation.ST_binary_evaluation(_, test_output_mort, 'xgb', mode = 'eicu', event_task = False)
display(test_eval)

,Model,Accuracy,AUPRC,F1 Score,Recall,Precision
0,xgb,0.463053,0.349561,0.438926,0.624528,0.338368


In [52]:
lgbm_wrapper = LGBMClassifier(random_state = 42, verbose=-1)
lgbm_wrapper.fit(X_train_los, y_train_los)

X_test_los, y_test_los, test_output_los = split.split_X_Y_LOS(los_eicu_test, mode = 'eicu')

X_test_los = X_test_los.drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)

test_preds_los = lgbm_wrapper.predict(X_test_los)
test_output_los['prediction_label'] = test_preds_los
test_output_los['prediction_prob'] = lgbm_wrapper.predict_proba(X_test_los)[:, 1]

test_eval_los = Multiclass_evaluation.ST_multi_evaluation(_, test_output_los, 'xgb')
display(test_eval_los)

,Model,Accuracy,Macro AUPRC,F1 Score,Macro averaged precision,Macro averaged Recall
0,xgb,0.6835,0.666667,0.5117,0.5729,0.5265


In [8]:
reload(Multiclass_evaluation) 

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [63]:
lgbm_wrapper = LGBMClassifier(random_state = 42, verbose=-1)
lgbm_wrapper.fit(X_train_ards8h, y_train_ards8h)

X_test_ards8h, y_test_ards8h, test_output_ards8h = split.split_X_Y_ARDS8h(ards8h_eicu_test, mode = 'eicu')

X_test_ards8h = X_test_ards8h.drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)

test_preds_ards8h = lgbm_wrapper.predict(X_test_ards8h)
test_output_ards8h['prediction_label'] = test_preds_ards8h
test_output_ards8h['prediction_prob'] = lgbm_wrapper.predict_proba(X_test_ards8h)[:, 1]

test_eval_ards8h = Multiclass_evaluation.ST_binary_evaluation(ARDS8h_event_test, test_output_ards8h, 'lgbm', mode='eicu', event_task='ARDS 8h')
display(test_eval_ards8h)

,Model,AUPRC,ARDS score,Recall,Precision,Grecall
0,lgbm,0.167527,0.179515,0.1769,0.226754,0.164303


In [73]:
len(X_train_ards8h)

24653

In [74]:
len(X_test_ards8h)

18059

In [62]:
lgbm_wrapper = LGBMClassifier(random_state = 42, verbose=-1)
lgbm_wrapper.fit(X_train_sic8h, y_train_sic8h)

X_test_sic8h, y_test_sic8h, test_output_sic8h = split.split_X_Y_SIC8h(sic8h_eicu_test, mode = 'eicu')

X_test_sic8h = X_test_sic8h.drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)

test_preds_sic8h = lgbm_wrapper.predict(X_test_sic8h)
test_output_sic8h['prediction_label'] = test_preds_sic8h
test_output_sic8h['prediction_prob'] = lgbm_wrapper.predict_proba(X_test_sic8h)[:, 1]

test_eval_sic8h = Multiclass_evaluation.ST_binary_evaluation(SIC8h_event_test, test_output_sic8h, 'lgbm', mode='eicu', event_task='SIC 8h')
display(test_eval_sic8h)

,Model,AUPRC,SIC score,Recall,Precision,Grecall
0,lgbm,0.363518,0.284738,0.255556,0.40658,0.251271


In [103]:
reload(Multiclass_evaluation) 

eicu_2014 = sic8h_eicu_test[sic8h_eicu_test['hospitaldischargeyear']==2014].drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)
eicu_2015 = sic8h_eicu_test[sic8h_eicu_test['hospitaldischargeyear']==2015].drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)


X_test_sic8h_2014, y_test_sic8h_2014, _ = split.split_X_Y_SIC8h(eicu_2014, mode = 'eicu')
X_test_sic8h_2015, y_test_sic8h_2015, test_output_sic8h = split.split_X_Y_SIC8h(eicu_2015, mode = 'eicu')


X = pd.concat([X_train_sic8h, X_test_sic8h_2014], axis = 0)
Y = np.concatenate((y_train_sic8h, y_test_sic8h_2014))


sic_models, result_sic = Multiclass_evaluation.create_subtask(X, Y, X_test_sic8h_2015, test_output_sic8h, SIC8h_event_test, mode='eicu', type='binary', event_task='SIC 8h')

|MIMIC-Test|====================================================


,Model,AUPRC,SIC score,Recall,Precision,Grecall
0,xgb,0.344747,0.284512,0.230769,0.467320,0.242373
1,lgbm,0.336351,0.235242,0.179487,0.340782,0.206780
2,catboost,0.339995,0.262081,0.192308,0.441077,0.222034
3,rf,0.326580,0.170157,0.051282,0.714286,0.127119
4,dt,0.309458,0.283462,0.410256,0.174868,0.391525
5,svm-ovr,0.260795,0.148771,0.089744,0.430380,0.115254
6,lr,0.211300,0.149954,0.141026,0.251678,0.127119
7,naivebayes,0.118577,0.234916,1.000000,0.086320,1.000000
8,knn,0.139003,0.159735,0.256410,0.173664,0.154237


----------------------------------------------------------------------
